In [1]:
from skimage.io import imread, imshow, imread_collection, concatenate_images
import matplotlib.pyplot as plt
import os
import sys
import random
import warnings

import numpy as np
from numpy import fliplr, flipud
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.optimizers import Adam
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.


2.0.0


In [2]:
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
 
def rle2mask(mask_rle, shape=(1600,256)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

In [3]:
train = pd.read_csv('../catalyst/data/train.csv')
IMAGE_PATH = r"../catalyst/data/train_images/"
TEST_IMAGE_PATH = r"../catalyst/data/test_images/"

In [4]:
IMG_HEIGHT = 256
IMG_WIDTH = 1600
IMG_CHANNELS = 3
DEFECT_CLASSES = 4
SCALE_FACTOR = 2
SAMPLE_SIZE = 940

In [5]:
train.head(20)

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...
5,0025bde0c.jpg,3,8458 14 8707 35 8963 48 9219 71 9475 88 9731 8...
6,0025bde0c.jpg,4,315139 8 315395 15 315651 16 315906 17 316162 ...
7,002af848d.jpg,4,290800 6 291055 13 291311 15 291566 18 291822 ...
8,002fc4e19.jpg,1,146021 3 146275 10 146529 40 146783 46 147038 ...
9,002fc4e19.jpg,2,145658 7 145901 20 146144 33 146386 47 146629 ...


In [6]:
from skimage import color
from skimage import io

class_indices = [[],[],[],[]]

for ind in range(train.shape[0]):
    if train['EncodedPixels'][ind] != '':
        class_indices[train['ClassId'][ind]-1].append(ind)
        
Y_train = np.zeros((SAMPLE_SIZE*4, int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), int(DEFECT_CLASSES)), dtype=np.bool)

sample_set = []
for defect_type in range(4):
    partial_sample_set = np.random.permutation(int(SAMPLE_SIZE/4))
    for sample_index in partial_sample_set:
        sample_set.append(class_indices[defect_type][sample_index])

n=0        
for ind in range(len(sample_set)):
    img = rle2mask(train['EncodedPixels'][sample_set[ind]])
    img = resize(img, (IMG_HEIGHT/SCALE_FACTOR, IMG_WIDTH/SCALE_FACTOR), mode='constant', preserve_range=True)
    Y_train[n,:,:,(train['ClassId'][sample_set[ind]]-1)] = img
    Y_train[n+1,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(img)
    Y_train[n+2,:,:,(train['ClassId'][sample_set[ind]]-1)] = flipud(img)
    Y_train[n+3,:,:,(train['ClassId'][sample_set[ind]]-1)] = fliplr(flipud(img))
    n+=4

In [7]:
len(class_indices[3])

801

In [8]:
X_train = np.zeros((SAMPLE_SIZE*4, int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), int(IMG_CHANNELS)), dtype=np.uint8)
n = 0
for ind in range(len(sample_set)):
    img = imread(IMAGE_PATH + train['ImageId'][sample_set[ind]])
    img = resize(img, (IMG_HEIGHT/SCALE_FACTOR, IMG_WIDTH/SCALE_FACTOR, IMG_CHANNELS), mode='constant', preserve_range=True)
    X_train[n,:,:,:] = img
    X_train[n+1,:,:,:] = fliplr(img)
    X_train[n+2,:,:,:] = flipud(img)
    X_train[n+3,:,:,:] = fliplr(flipud(img))
    n += 4

In [9]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [10]:
inputs = Input((int(IMG_HEIGHT/SCALE_FACTOR), int(IMG_WIDTH/SCALE_FACTOR), IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.2) (c1)
c1 = BatchNormalization()(c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
c1 = BatchNormalization()(c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.3) (c2)
c2 = BatchNormalization()(c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
c2 = BatchNormalization()(c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.3) (c3)
c3 = BatchNormalization()(c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
c3 = BatchNormalization()(c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.4) (c4)
c4 = BatchNormalization()(c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
c4 = BatchNormalization()(c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = Dropout(0.5) (c5)
c5 = BatchNormalization()(c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
u6 = BatchNormalization()(u6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.4) (c6)
c6 = BatchNormalization()(c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)
c6 = BatchNormalization()(c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
u7 = BatchNormalization()(u7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.3) (c7)
c7 = BatchNormalization()(c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
u8 = BatchNormalization()(u8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.3) (c8)
c8 = BatchNormalization()(c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)
c8 = BatchNormalization()(c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
u9 = BatchNormalization()(u9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.2) (c9)
c9 = BatchNormalization()(c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)
c9 = BatchNormalization()(c9)

outputs = Conv2D(4, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
adamcustom = Adam(lr=0.0003)
model.compile(optimizer=adamcustom, loss=['binary_crossentropy'], metrics=[dice_coef])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 800, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 128, 800, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 800, 16) 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 128, 800, 16) 0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [11]:
earlystopper = EarlyStopping(patience=8, verbose=1)
results = model.fit(X_train, Y_train, validation_split=0.1, shuffle=True, batch_size=30, epochs=100, callbacks=[earlystopper])

Train on 3384 samples, validate on 376 samples
Epoch 1/100


ResourceExhaustedError:  OOM when allocating tensor with shape[30,64,64,400] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradients/batch_normalization_15/cond_grad/If/then/_728/zeros_like}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_22334]

Function call stack:
keras_scratch_graph


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from tensorflow.keras import backend as K
K.tensorflow_backend._get_available_gpus()